In [1]:
import sys
sys.path.append('../src/')

%load_ext autoreload

%autoreload 2

from utils import load_dataset, clean_dataset

In [2]:
df = load_dataset()

100%|██████████| 23/23 [00:10<00:00,  2.24it/s]


In [3]:
df_clean = clean_dataset(df)

In [4]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1053919 entries, 0 to 1053918
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   PublishYear   1053919 non-null  int64  
 1   Publisher     1053919 non-null  object 
 2   Rating        1053919 non-null  float64
 3   PagesNumber   1053919 non-null  float64
 4   TotalReviews  1053919 non-null  int64  
 5   Description   1053919 non-null  object 
dtypes: float64(2), int64(2), object(2)
memory usage: 48.2+ MB


In [5]:
df_clean.head(3)

,PublishYear,Publisher,Rating,PagesNumber,TotalReviews,Description
0,1993,[o'reilly media],0.0,458.0,0,"these days, it's a rare person who hasn't hear..."
1,2006,[casa creación],4.0,254.0,2,<i>¡aprende a caminar en el poder de la fe vio...
2,1999,[Unknown],2.8,514.0,5,this three-level program brings you every reso...


In [6]:
df_clean.index.size

1053919

In [7]:
df_clean.to_csv('../data/clean_dataset.csv')